
---
## Projectr : 스마트서울 도시데이터 센서(S-DoT) 유동인구 측정 정보분석 ver.01
### Description : 
    - 스마트서울 도시데이터 센서(S-DoT) 유동인구 측정 정보
    [source link](https://data.seoul.go.kr/dataList/OA-15964/S/1/datasetView.do)
    - 센서수량 및 설치장소 : 서울 전역 100대(전통시장, 주요거리, 공원, 공공시설)
    - 측정정보 : S-DoT에서 측정된 유동인구(10분단위 데이터)
    - 유동인구 센서는 WiFi AP셀 반경내 휴대폰의 MAC주소를 카운팅(50대)하거나 CCTV를 활용한 피플카운팅(50대) 방식으로 정확한 유동인구를 측정한 것이 아닙니다.
    - 데이터의 양이 많아 2023년 10월 16일부터 Sheet와 OpenAPI는 최근 한달만 출력됩니다.
### Author : Forrest Dpark (분석 담당)
### Date : 2024.05.31 ~
### Detail :  
### Update: 
    > ※ 2024.06.03 pdg : 데이터 체크 
        - api 연결
---

In [24]:
import pandas as pd ,numpy as  np # Data manipulatioin
import seaborn as sns,matplotlib.pyplot as plt # visiulization
import warnings;warnings.filterwarnings('ignore') # 경고 무시 
# 기본 세팅
def plotSetting(pltStyle):
    import matplotlib.pyplot as plt # visiulization
    import platform
    from matplotlib import font_manager, rc # rc : 폰트 변경 모듈font_manager : 폰트 관리 모듈
    plt.style.use(pltStyle)
    plt.rcParams['axes.unicode_minus'] = False# unicode 설정
    if platform.system() == 'Darwin': rc('font', family='AppleGothic') # os가 macos
    elif platform.system() == 'Windows': # os가 windows
        path = 'c:/Windows/Fonts/malgun.ttf' 
        font_name = font_manager.FontProperties(fname=path).get_name()
        rc('font', family=font_name)
    else:
        print("Unknown System")
    print("___## OS platform 한글 세팅완료 ## ___")
# graph style seaborn
plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___


In [155]:
def JsonDataFetch(basicURL,key,data_type,start,end):
    import urllib.request as request
    import json
    # 기상청 Rss 사이트 주소 
    query  = f"/{key}/{data_type}/IotVdata018/{start}/{end}/"
    openAPI_Address= basicURL + query
    
    # key ="646b77636970756c37314f63727871"
    # openAPI_Address = f"http://openapi.seoul.go.kr:8088/{key}/json/IotVdata018/1/2/"
    # openAPI_Address = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/SPOP_ORGN_CT_IN_SE_DONG/1/5/"
    # URL 생성 
    print(openAPI_Address)
    # 다운로드 
    data = request.urlopen(openAPI_Address).read()
    print(data) ## utf 8 에 xml .. 옛날 방식 

    text = data.decode("UTF-8")
    items = json.loads(text)


    print(items)
    return items
items = JsonDataFetch(
            basicURL="http://openapi.seoul.go.kr:8088",
            key="646b77636970756c37314f63727871",
            data_type ='json',
            start = 1,
            end = 100
            )



http://openapi.seoul.go.kr:8088/646b77636970756c37314f63727871/json/IotVdata018/1/100/
b'{"IotVdata018":{"list_total_count":351903,"RESULT":{"CODE":"INFO-000","MESSAGE":"\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4"},"row":[{"MODEL_NM":"SDOT001","SERIAL_NO":"00000003005","SENSING_TIME":"2024-06-02_23:44:00","REGION":"traditional_markets","AUTONOMOUS_DISTRICT":"Gangnam-gu","ADMINISTRATIVE_DISTRICT":"Yeoksam2-dong","VISITOR_COUNT":"220","REG_DTTM":"2024-06-02 23:58:05"},{"MODEL_NM":"SDOT001","SERIAL_NO":"00000003039","SENSING_TIME":"2024-06-02_23:44:00","REGION":"traditional_markets","AUTONOMOUS_DISTRICT":"Dongjak-gu","ADMINISTRATIVE_DISTRICT":"Sadang2-dong","VISITOR_COUNT":"181","REG_DTTM":"2024-06-02 23:58:05"},{"MODEL_NM":"SDOT001","SERIAL_NO":"00000003036","SENSING_TIME":"2024-06-02_23:44:00","REGION":"traditional_markets","AUTONOMOUS_DISTRICT":"Dongjak-gu","ADMINISTRATIVE_DISTRICT":"Sang-do4-dong","VISITOR_COUNT":"279"

In [101]:
def ifDictOrArray(data):
    if type(data)==type({}):
        print("Dict 데이터 입니다. ")
        for i in data.keys():
            print(i, end="")
        for k,v in data.items():
            print(k,v)

    elif type(data) == type([]):
        print("Array 데이터 입니다. ")
        print(f"|- >arraysize = {data.shape} ")
        for i in data:
            print("|- ",i)

def jsonDataProcessing(items):
    if type(items)== type({}): # dictionary 타입일경우 
        print(" |- [1차 트리노드] :",items.keys()) # IotVdata018
        for idx , key in enumerate(list(items.keys())):
            print(f"  * {idx} 번째 노드: '{key}'",)
            # print(items[key])
            print(" |-","-"*5,f"[2차 트리노드] :{key} 에 있는 데이터","-"*5,f"\n\t=>{list(items[key].keys())}")
            
            if type(items[key])==type({}): ## 2차 트리가 dict 인경우
                print("\t\t  |-","-"*5,f"[3차 트리노드] :" ,sep="")
                
                for j, subkey in enumerate(items[key].keys()):
                    print("\t\t  |-","-"*5,f"[{key}][{subkey}] :" ,sep="")
                    print(f"\t\t\t  # content : {items[key][subkey]} \
                          \n\t\t\t  # type : {type(items[key][subkey])}")
                    
                    if type(items[key][subkey])== type({}):
                        
                    
                    ## 3차 
            else : 
                print("배열 타입입니다. ")
                print(items[key[0]])
    else: ## 배열일경우 첫 인덱스를 뽑는다. 
        jsonDataProcessing(items[0])
        


jsonDataProcessing(items)


 |- [1차 트리노드] : dict_keys(['IotVdata018'])
  * 0 번째 노드: 'IotVdata018'
 |- ----- [2차 트리노드] :IotVdata018 에 있는 데이터 ----- 
	=>['list_total_count', 'RESULT', 'row']
		  |------[3차 트리노드] :
		  |------[IotVdata018][list_total_count] :
			  # content : 351903                           
			  # type : <class 'int'>
		  |------[IotVdata018][RESULT] :
			  # content : {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}                           
			  # type : <class 'dict'>
		  |------[IotVdata018][row] :
			  # content : [{'MODEL_NM': 'SDOT001', 'SERIAL_NO': '00000003005', 'SENSING_TIME': '2024-06-02_23:44:00', 'REGION': 'traditional_markets', 'AUTONOMOUS_DISTRICT': 'Gangnam-gu', 'ADMINISTRATIVE_DISTRICT': 'Yeoksam2-dong', 'VISITOR_COUNT': '220', 'REG_DTTM': '2024-06-02 23:58:05'}, {'MODEL_NM': 'SDOT001', 'SERIAL_NO': '00000003039', 'SENSING_TIME': '2024-06-02_23:44:00', 'REGION': 'traditional_markets', 'AUTONOMOUS_DISTRICT': 'Dongjak-gu', 'ADMINISTRATIVE_DISTRICT': 'Sadang2-dong', 'VISITOR_COUNT': '18

> api 의 data 가 json 일때 까도까도 dict 인 경우 실제로 필요한 데이터를 찾기 위해 tree 구조를 제작하여 
데이터를 확인하도록 함수를 만든다. 


In [156]:

def jsonTreeCheck(items):
    def ifDict_ReturnKeyList(data,treeNum):
        # print((type(data) != type([])))
        # print((type(data) != type({})))
        
        indentSize =2 
        indentation = " "*indentSize
        print(indentation*treeNum, f"*{treeNum}th")
        if (type(data) != type([])) and (type(data) != type({})):
            print(indentation*(treeNum+1),data)
        if type(data)==type({}):
            # print(" "*treeNum,"Dict 데이터 입니다. ")
            if len(list(data.keys()))==1:
                print(indentation*(treeNum+1),f"|-> {list(data.keys())}")
            else :
                for key in data.keys():
                    print(indentation*(treeNum+1),f"|-> ['{key}'] ({type(data[key])})")
                    ifDict_ReturnKeyList(data[key],3)
                    
            # print(" "*treeNum,"|- key list type 으로 return")
            return list(data.keys())

        elif type(data) == type([]):
            print("Array 데이터 입니다. ")
            print(f"|- >arraysize =  ")
            for i in data:
                print("|- ",i)
    ## 1st tree
    item_key_list=ifDict_ReturnKeyList(items,1)
    
    ## 2nd tree
    for key in item_key_list:
        # print(items[key])   
        ifDict_ReturnKeyList(items[key],2)
jsonTreeCheck(items)

   *1th
     |-> ['IotVdata018']
     *2th
       |-> ['list_total_count'] (<class 'int'>)
       *3th
         351903
       |-> ['RESULT'] (<class 'dict'>)
       *3th
         |-> ['CODE'] (<class 'str'>)
       *3th
         INFO-000
         |-> ['MESSAGE'] (<class 'str'>)
       *3th
         정상 처리되었습니다
       |-> ['row'] (<class 'list'>)
       *3th
Array 데이터 입니다. 
|- >arraysize =  
|-  {'MODEL_NM': 'SDOT001', 'SERIAL_NO': '00000003005', 'SENSING_TIME': '2024-06-02_23:44:00', 'REGION': 'traditional_markets', 'AUTONOMOUS_DISTRICT': 'Gangnam-gu', 'ADMINISTRATIVE_DISTRICT': 'Yeoksam2-dong', 'VISITOR_COUNT': '220', 'REG_DTTM': '2024-06-02 23:58:05'}
|-  {'MODEL_NM': 'SDOT001', 'SERIAL_NO': '00000003039', 'SENSING_TIME': '2024-06-02_23:44:00', 'REGION': 'traditional_markets', 'AUTONOMOUS_DISTRICT': 'Dongjak-gu', 'ADMINISTRATIVE_DISTRICT': 'Sadang2-dong', 'VISITOR_COUNT': '181', 'REG_DTTM': '2024-06-02 23:58:05'}
|-  {'MODEL_NM': 'SDOT001', 'SERIAL_NO': '00000003036', 'SENSING_TIME': 

> 실제 데이터는 items['IotVdata018']['row'] 에 있음

In [163]:
def targetDataFetching(*arg):
    for row, c in enumerate(items[arg[0]][arg[1]]):
        #print(row)
        region =items['IotVdata018']['row'][row]['REGION']
        sensingTime =items['IotVdata018']['row'][row]['SENSING_TIME']
        district =items['IotVdata018']['row'][row]['AUTONOMOUS_DISTRICT']
        admin_distric =items['IotVdata018']['row'][row]['ADMINISTRATIVE_DISTRICT']
        visit_count =items['IotVdata018']['row'][row]['VISITOR_COUNT']
        print("-"*40)
        print(district,admin_distric)
        print(region)
        print(sensingTime)
        print(admin_distric)
        print(visit_count)
targetDataFetching('IotVdata018','row')

----------------------------------------
Gangnam-gu Yeoksam2-dong
traditional_markets
2024-06-02_23:44:00
Yeoksam2-dong
220
----------------------------------------
Dongjak-gu Sadang2-dong
traditional_markets
2024-06-02_23:44:00
Sadang2-dong
181
----------------------------------------
Dongjak-gu Sang-do4-dong
traditional_markets
2024-06-02_23:44:00
Sang-do4-dong
279
----------------------------------------
Dongjak-gu Sadang2-dong
traditional_markets
2024-06-02_23:44:00
Sadang2-dong
154
----------------------------------------
Dongjak-gu Sang-do1-dong
traditional_markets
2024-06-02_23:44:00
Sang-do1-dong
231
----------------------------------------
Gangnam-gu Yeoksam1-dong
main_street
2024-06-02_23:45:00
Yeoksam1-dong
55
----------------------------------------
Gangnam-gu Sinsa-dong
main_street
2024-06-02_23:40:00
Sinsa-dong
4
----------------------------------------
Seocho-gu Seocho3-dong
main_street
2024-06-02_23:40:00
Seocho3-dong
10
----------------------------------------
Dongjak-

'MODEL_NM':모델번호,    
'SERIAL_NO':시리얼',    
'SENSING_TIME': 측정시간    
'REGION': 지역  
'AUTONOMOUS_DISTRICT':자치구,   
'ADMINISTRATIVE_DISTRICT'행정동:     
'VISITOR_COUNT':방문자수,   
'REG_DTTM': 등록일  
> 측정시간 , 지역, 자치구, 행정동, 방문자수 를 컬럼으로 뽑는다


In [150]:
region =items['IotVdata018']['row'][0]['REGION']
sensingTime =items['IotVdata018']['row'][0]['SENSING_TIME']
district =items['IotVdata018']['row'][0]['AUTONOMOUS_DISTRICT']
admin_distric =items['IotVdata018']['row'][0]['ADMINISTRATIVE_DISTRICT']
visit_count =items['IotVdata018']['row'][0]['VISITOR_COUNT']

print(region)
print(sensingTime)
print(district)
print(admin_distric)
print(visit_count)


traditional_markets
2024-06-02_23:44:00
Gangnam-gu
Yeoksam2-dong
220
